In [1]:
! pip install pandas numpy

In [2]:
import pandas as pd
import numpy as np
from ast import literal_eval
import re
from ast import literal_eval

In [ ]:
# Word segmenter
!pip3 install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows

In [6]:
train = pd.read_csv('train.csv')
dev = pd.read_csv('dev.csv')
test = pd.read_csv('test.csv')
test['index_spans'] = test['index_spans'].apply(literal_eval)
train['index_spans'] = train['index_spans'].apply(literal_eval)
dev['index_spans'] = dev['index_spans'].apply(literal_eval)

headers = ['Unnamed: 0',  'content', 'index_spans', 'Span']
train.columns = headers
dev.columns = headers
test.columns = headers
test.head(2)

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

# Pre-processing

In [ ]:
def unicode(data):
  for i in range(len(data)):
    data['content'][i] = data['content'][i].replace("òa", "oà")
    data['content'][i] = data['content'][i].replace("óa", "oá")
    data['content'][i] = data['content'][i].replace("ỏa", "oả")
    data['content'][i] = data['content'][i].replace("õa", "oã")
    data['content'][i] = data['content'][i].replace("ọa", "oạ")
    data['content'][i] = data['content'][i].replace("òe", "oè")
    data['content'][i] = data['content'][i].replace("óe", "oé")
    data['content'][i] = data['content'][i].replace("ỏe", "oẻ")
    data['content'][i] = data['content'][i].replace("õe", "oẽ")
    data['content'][i] = data['content'][i].replace("ọe", "oẹ")
    data['content'][i] = data['content'][i].replace("ùy", "uỳ")
    data['content'][i] = data['content'][i].replace("úy", "uý")
    data['content'][i] = data['content'][i].replace("ủy", "uỷ")
    data['content'][i] = data['content'][i].replace("ũy", "uỹ")
    data['content'][i] = data['content'][i].replace("ụy", "uỵ")
    data['content'][i] = data['content'][i].replace("Ủy", "Uỷ")
  return data
def replace(text, pattern, replacement, pos):
    matches = [0]

    def capture_and_replace(match, ret):
        matches.extend([match.start() + 1, match.end()])
        return ret

    l = len(text)
    text = re.sub(pattern, lambda match: capture_and_replace(match, replacement), text, flags=re.IGNORECASE)
    matches.append(l)
    slices = np.array_split(matches, int(len(matches) / 2))
    res = []
    for s in slices:
        res += pos[s[0]:s[1]]
    assert len(text) == len(res)
    return text, res
def preprocess(text, pos):
    
    # collapse duplicated punctuations 
    punc = ',. !?\"\''
    for c in punc:
        pat = '([' + c + ']{2,})'
        text, pos = replace(text, pat, c, pos)
    assert len(text) == len(pos)
    return text, pos

test = unicode(test)
train = unicode(train)
dev = unicode(dev)

# Tokeniner

In [ ]:
df_test = pd.DataFrame()
df_test['spans'] = test['index_spans']
df_test['text'] = test['content']
df_test.to_csv('df_test.csv')

df_train = pd.DataFrame()
df_train['spans'] = train['index_spans']
df_train['text'] = train['content']
df_train.to_csv('df_train.csv')

df_dev = pd.DataFrame()
df_dev['spans'] = dev['index_spans']
df_dev['text'] = dev['content']
df_dev.to_csv('df_dev.csv')

In [ ]:
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

In [ ]:
text = test['content'][325]
text = 'Nham quá'
annotator_text = annotator.tokenize(text)
tokens = []
for i in range(len(annotator_text)):
  for j in range(len(annotator_text[i])):
    tokens.append(annotator_text[i][j])
tokens

In [ ]:
import more_itertools as mit
from itertools import groupby
from operator import itemgetter
def find_ranges(span):
  ranges =[]
  for k,g in groupby(enumerate(span),lambda x:x[0]-x[1]):
      group = (map(itemgetter(1),g))
      group = list(map(int,group))
      ranges.append((group[0],group[-1]))
  return ranges
def tokenize_word(text, pos):
    annotator_text = annotator.tokenize(text)
    tokens = []
    for i in range(len(annotator_text)):
      for j in range(len(annotator_text[i])):
        tokens.append(annotator_text[i][j])

    alignment = []
    start = 0
    for t in tokens:
      if t == "_":    
        res = text.find(t, start)
        alignment.append(pos[res:res + len(t)])
        start = res + len(t)
      elif t.find("_.", 0) != -1 or t.find("_-", 0) != -1:       # k xuất hiện _.
        t = t.replace("_","")
        res = text.find(t, start)
        alignment.append(pos[res:res + len(t)])
        start = res + len(t)
      else:
        t = t.lstrip("_")
        t = t.replace("_", " ")
        res = text.find(t, start)
        alignment.append(pos[res:res + len(t)])
        start = res + len(t)       
    assert len(tokens) == len(alignment)
    return tokens, alignment
def annotate(spans, alignment, tokens):
  Tag = []
  annotations = pd.DataFrame()
  annotations['Tokens'] = tokens
  for i in range(len(tokens)):
    Tag.append("O")
  annotations['Tag'] = Tag
  for span in spans:
      i = 0
      while i < len(alignment):
          if alignment[i][-1] < span[0]:
            i += 1
          elif alignment[i][0] <= span[0] <= alignment[i][-1]:
              annotations['Tag'][i] = ('B-T')
              i += 1
          elif span[0] < alignment[i][0] <= span[-1]:
              annotations['Tag'][i] = ('I-T')
              i += 1
          elif alignment[i][0] > span[-1]:
              break
  return annotations['Tag']
def load_data(path):
  tsd = pd.read_csv(path)
  tsd.spans = tsd.spans.apply(literal_eval)
  data = []
  for row in tsd.iterrows():
      span = row[1]['spans']
      text = row[1]['text']
      temp = []
      text_spans = []
      if span:
        segments = list(find_ranges(span))
        for seg in segments:
          if ((len(seg) == 2) and seg[0] == seg[1]):
            temp.append([seg[0]])
            text_spans.append(text[seg[0]: seg[-1] + 1])
          else:
            temp.append([seg[0], seg[-1]])
            text_spans.append(text[seg[0]: seg[-1] + 1])
      data.append({'text': text, 'spans': temp, 'text_spans': text_spans}) 
  return data

In [ ]:
test_data = load_data('df_test.csv')
train_data = load_data('df_train.csv')
dev_data = load_data('df_dev.csv')


# Annotate BIO

In [ ]:
def annotate(spans, alignment, tokens):
  Tag = []
  annotations = pd.DataFrame()
  annotations['Tokens'] = tokens
  for i in range(len(tokens)):
    Tag.append("O")
  annotations['Tag'] = Tag
  for span in spans:
      i = 0
      while i < len(alignment):
          if alignment[i][-1] < span[0]:
            i += 1
          elif alignment[i][0] <= span[0] <= alignment[i][-1]:
              annotations['Tag'][i] = ('B-T')
              i += 1
          elif span[0] < alignment[i][0] <= span[-1]:
              annotations['Tag'][i] = ('I-T')
              i += 1
          elif alignment[i][0] > span[-1]:
              break
  return annotations['Tag']

In [ ]:
def data_BIO(data):
  formated_data = []
  for d in data:
    text = d['text']
    pos = [i for i in range(len(text))]
    text, pos = preprocess(text, pos)
    tokens, alignment = tokenize_word(text, pos)
    annotations = annotate(d['spans'], alignment, tokens)
    ls = [[tokens[i], annotations[i]] for i in range(len(tokens))]
    
    formated_data.extend(ls)
    formated_data.append([None])
  df_final =  pd.DataFrame(formated_data, columns= ['Word', 'Tag'])
  sentence_id = []
  sentence = 0
  for i in range(len(df_final)):
    if(df_final['Word'][i] != None):
      sentence_id.append(sentence)
    else:
      sentence_id.append(np.nan)
      sentence += 1
  df_final['sentence_id'] = sentence_id
  df_final.dropna(inplace=True)
  df_final['sentence_id'] = df_final['sentence_id'].astype("int64")
  return df_final




In [ ]:
test_IBO = data_BIO(test_data)
train_IBO = data_BIO(train_data)
dev_IBO = data_BIO(dev_data)
test_IBO

In [ ]:
test_IBO.reset_index(inplace=True)
dev_IBO.reset_index(inplace=True)
train_IBO.reset_index(inplace=True)

In [ ]:
train_IBO.to_csv('train_BIO.csv')
dev_IBO.to_csv('dev_BIO.csv')
test_IBO.to_csv('test_BIO.csv')